# FIT5202 — Week 4a & 4b Notes (Parallel Sort & GroupBy)

---

## Week 4a — Parallel Sort

### Internal vs External Sorting
- **Internal sort**: data fits in RAM. Examples: Bubble Sort, Quick Sort.
- **External sort**: data too large for RAM → use **sort-merge**:
  1. Form runs (Pass 0) - Break the file up into unsorted subfiles, then sort the subfiles.
  2. Repeatedly merge runs until sorted.

---

### Serial External Sort (examples from slides)

- **Initial runs (Example 1)**  
  $$
  \#\text{runs} = 108/5 = 22
  $$
  With 108 pages and 5 buffer pages, we form 22 sorted runs.

- **Merging buffers**  
  $$
  \text{input buffers} = B-1,\quad \text{output buffers} = 1
  $$
  With $B=5$ buffers we can merge 4 runs at a time.

- **Pass count (Example 1)**  
  $$
  \text{passes} = 4
  $$
  → Pass 0 (run generation) + 3 merge passes.

- **Buffer size effect (Example 2)**  
  $$
  4\times 8 + 6 = 38\ \text{pages}
  $$
  Shows how run merging arithmetic works when $R=150$ pages and $B=8$ buffers.

**Variable notes**  
- $B$: number of buffer pages in memory.  
- “page”: disk page (fixed-size I/O unit).  
- “run”: sorted subfile created in Pass 0.  
- “pass”: one full sweep/merge phase of the runs.

---

### Parallel External Sort — Algorithms

- **Parallel Merge-All Sort**: each node sorts locally, then one node merges *everything*.
  - *Issue*: bottleneck at the coordinator, high network contention.

- **Parallel Binary-Merge Sort**: pairwise merges in a pipeline across processors.
  - *Issue*: longer pipeline, more passes.

- **k-way vs Binary merging**:
  - **k-way**: compare $k$ runs at once (requires many open files/buffers).
  - **Binary**: compare 2 at a time (longer merge sequence).

- **Other methods listed**: Redistribution Binary-Merge, Redistribution Merge-All, Partitioned Sort.

---

### Thought-provoking questions (Sort)

- **Q: Why is Merge-All a bottleneck?**  
  **A:** Because all intermediate results and network transfers funnel into one node, causing a hotspot.

- **Q: When is Binary Merge preferable to k-way?**  
  **A:** When memory is too small to hold $k$ input buffers or open many runs simultaneously.

---

## Week 4b — Parallel GroupBy

### Serial GroupBy (hashing in memory)
- Records are hashed into a hash table keyed by group attribute.
- Aggregates (e.g. COUNT, SUM) are updated bucket by bucket.

---

### Parallel GroupBy — Methods

1. **Traditional “Merge-All”**  
   - Local partial aggregates first.  
   - One coordinator gathers and merges everything.  
   - *Limitations*: single node does final work, network bottleneck, no parallelism in global stage.

   **Exercise (slide question): What are its limitations?**  
   **Answer:** All of the above — bottleneck, single node workload, no parallelism.

---

2. **Two-Phase Method**  
   - **Phase 1**: local aggregate on each processor.  
   - **Phase 2**: redistribute these condensed results by group key, then final aggregation in parallel.

---

3. **Redistribution Method**  
   - **Step 1**: redistribute **raw** tuples by group key (partitioning).  
   - **Step 2**: each processor aggregates its partition.

   **Slide prompt: “What is the problem here?”**  
   **Answer:** Skew / load imbalance — some partitions receive many more keys/tuples.

   **Load-balancing fix**:  
   - Over-partition into more buckets than processors.  
   - Allow *task stealing*: idle processors take buckets from overloaded ones.

---

### Summary rule of thumb (from lecture)
- **Two-Phase**: good when number of groups is **small** (local aggregation shrinks data a lot before shuffle).  
- **Redistribution**: good when number of groups is **large** (local aggregation shrinks little, direct partitioning distributes work more evenly).

---

### Thought-provoking questions (GroupBy)

- **Q: Why Two-Phase for few groups, but Redistribution for many groups?**  
  **A:** Few groups → heavy reduction locally, so network cost is small and final aggregation balanced.  
  Many groups → little reduction locally, so better to partition raw data evenly from the start.

- **Q: Where can super-linear speedup occur?**  
  **A:** If parallelization lets each processor’s hash table fit in memory (while serial spills), I/O is avoided and speedup > N.

- **Q: True/False: Redistribution has a load-balancing option (task stealing), Two-Phase has no load-balancing problem.**  
  **A:** False. Redistribution explicitly uses task stealing, but Two-Phase can still face skew if group key distribution is uneven.

---

## Quick glossary (this week’s variables)

- $B$: number of buffer pages (for sort).  
- “page”: disk I/O unit.  
- “run”: sorted subfile created during external sort.  
- “pass”: one complete round of sorting/merging.  
- $k$: degree of k-way merge.  
- “group key”: attribute(s) in `GROUP BY`.  
- “local aggregation”: per-processor partial grouping.  
- “redistribution”: network shuffle by group key.  
- “task stealing”: reassigning buckets to balance load.


Sorting and Serial Sorting
Serial Sorting - Internal
- Data fits entirely into main memory
    - Bubble sort
    - Insertion sort
    - Quick sort

Serial Sorting - External
- Data does NOT fit entirely into main memory

Sort by is to sort a result by list of column - allows duplication
Group by is for you to create a unique combination of group of columns - unique columns only